In [ ]:
!pip install transformers

In [35]:
import pandas as pd
data = pd.read_excel('/content/База стартапов_.xlsx')
data.dropna(subset=['Что делают', 'Подробное описание проекта'], inplace=True)
data

,Название компании,Рынок,Что делают,Подробное описание проекта,Выручка (млн),Стадия,Международный трекшн,Полезные ссылки
0,Unspot,Business Software,Система для бронирования офисного пространства,Система UnSpot – это программное обеспечение д...,"Lj 100 vky/he,",Seed,Да,Сайт: https://unspot.com/\n\n\nСсылка на Сбис:...
1,Реокат,IndustrialTech,Производят катализаторы для нефтехимии,Компания ООО «РеоКат» работает в области серви...,100-500 млн.руб.,Раунд А,Нет,Сайт: https://reocat.ru/\n\n\n\nСсылка на Сбис...
2,Workface,E-commerce,Онлайн конструктор для интернет магазина,Компания специализируется на разработке прилож...,100-500 млн.руб.,Раунд B,Да,NaN
3,Аллока,Advertising & Marketing,Программа отслеживания рекламы с сайта.,Программа отслеживания рекламы с сайта. \nКолл...,До 100 млн.руб,Pre-seed,Нет,Сайт: \nhttps://alloka.ru/\n\nСбис: \nhttps://...
4,Инновации Детям,Hardware,Создают итерактивное оборудование для обучения...,Playstand — производитель интеректавно оборуд...,100-500 млн.руб.,Раунд А,Да,Сайт: \nhttps://playstand.ru/contacts/;\n\nСби...
...,...,...,...,...,...,...,...,...
196,Joys,FinTech,Сервис для оплаты по QR-кодам,Сервис для оплаты по QR-кодам,До 100 млн.руб,Pre-seed,Нет,NaN
197,Everytale,Media & Entertainment,Сервис для проведения онлайн-мероприятий,Сервис для проведения онлайн-мероприятий,До 100 млн.руб,Pre-seed,Нет,NaN
198,imin,Advertising & Marketing,Конструктор видеоконтента для социальных сетей,Конструктор видеоконтента для социальных сетей,До 100 млн.руб,Pre-seed,Нет,NaN
199,ARS Smart Robotics,FoodTech,Система автоматизации для дарксторов,Система автоматизации для дарксторов,До 100 млн.руб,Pre-seed,Нет,NaN


In [23]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np
model_name = 'Skoltech/russian-sensitive-topics'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

qa_pipeline = pipeline(
    "question-answering",
    model="AndrewChar/model-QA-5-epoch-RU",
    tokenizer="AndrewChar/model-QA-5-epoch-RU"
)

Some layers from the model checkpoint at AndrewChar/model-QA-5-epoch-RU were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at AndrewChar/model-QA-5-epoch-RU and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def get_seq_embed(seq):
    tokenized = tokenizer.batch_encode_plus([seq],max_length = 512,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False)
    tokens_ids,mask = torch.tensor(tokenized['input_ids']),torch.tensor(tokenized['attention_mask'])
    with torch.no_grad():
        model_output = model(tokens_ids,mask)
    def adjust_multilabel(y, is_pred = False):
        y_adjusted = []
        return y
    preds = adjust_multilabel(model_output['logits'], is_pred = True)
    return preds

In [22]:
from sklearn.metrics import mean_squared_error

seq1 = get_seq_embed('Я люблю пиво Толстяк и пью его каждый день')
seq2 = get_seq_embed('Дайте денег за цп очень надо голодаю')
seq3 = get_seq_embed('Я предпочитаю пиво балтика девятка и пью его по пятницам с Валерой и Саньком')
print(mean_squared_error(seq1, seq2))
print(mean_squared_error(seq1, seq3))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1.3357638
0.4236948


In [29]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
mse_list = []

for i in range(data.shape[0]):
    context = data.iloc[i]['Подробное описание проекта']
    expected = data.iloc[i]['Что делают']

    prediction = qa_pipeline({
                    'context': context,
                    'question': "Какая сфера деятельности у компании?"
    })['answer']

    seq1 = get_seq_embed(expected)
    seq2 = get_seq_embed(prediction)
    mse = mean_squared_error(seq1, seq2)
    mse_list.append(mse)
    print(expected, '~', prediction, '~', mse)

Система для бронирования офисного пространства ~ устойчивый рост финансовых показателей ~ 0.17970738
Производят катализаторы для нефтехимии ~ на релизации проектов ~ 0.099153675
Онлайн конструктор для интернет магазина ~ действует на рынке США ~ 0.14446692
Программа отслеживания рекламы с сайта. ~ Программа отслеживания рекламы с сайта ~ 0.0030377244
Создают итерактивное оборудование для обучения детей ~ разнообразные игры и развлекательные сервисы ~ 1.1428752
Платформа управления документами с помощью AI. ~ кейсы ~ 0.03380759
Аналитика полочного пространства ~ FMCG, Фармы и Ритейла ~ 0.04063561
Инновационная технология производства спг ~ уникальная технология российского производства ~ 0.40466306
Инновационная технология производства спг ~ провела эксперименты, предварительные расчёты ~ 0.9978241
Инновационная технология обеззараживания воды ~ инновационная разработка ~ 0.080108024
Инновационная технология обеззараживания воды ~ уникальная технология ~ 0.043778908
Получение сверхчисто

In [38]:
sum(mse_list) / len(mse_list)

0.16901302389607112